# EDA Dataset Hotels

## Dataset

Ce jeu de données est issu du site tidymodels.

Je reprends leur description :
"Let’s use hotel bookings data from Antonio, Almeida, and Nunes (2019) to predict which hotel stays included children and/or babies, based on the other characteristics of the stays such as which hotel the guests stay at, how much they pay, etc. This was also a #TidyTuesday dataset with a data dictionary you may want to look over to learn more about the variables. We’ll use a slightly edited version of the dataset for this case study."



#### Import du jeu de données

In [1]:
import sys
sys.path.append("../scripts")   # car ton notebook est dans notebooks/

import utils

import pandas as pd

df = pd.read_csv("../data/raw/hotels.csv")
print(df.shape)
df.head()

(50000, 23)


,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,market_segment,distribution_channel,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,average_daily_rate,required_car_parking_spaces,total_of_special_requests,arrival_date
0,City_Hotel,217,1,3,2,none,BB,DEU,Offline_TA/TO,TA/TO,...,A,A,0,No_Deposit,0,Transient-Party,80.75,none,1,2016-09-01
1,City_Hotel,2,0,1,2,none,BB,PRT,Direct,Direct,...,D,K,0,No_Deposit,0,Transient,170.00,none,3,2017-08-25
2,Resort_Hotel,95,2,5,2,none,BB,GBR,Online_TA,TA/TO,...,A,A,2,No_Deposit,0,Transient,8.00,none,2,2016-11-19
3,Resort_Hotel,143,2,6,2,none,HB,ROU,Online_TA,TA/TO,...,A,A,0,No_Deposit,0,Transient,81.00,none,1,2016-04-26
4,Resort_Hotel,136,1,4,2,none,HB,PRT,Direct,Direct,...,F,F,0,No_Deposit,0,Transient,157.60,none,4,2016-12-28


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           50000 non-null  object 
 1   lead_time                       50000 non-null  int64  
 2   stays_in_weekend_nights         50000 non-null  int64  
 3   stays_in_week_nights            50000 non-null  int64  
 4   adults                          50000 non-null  int64  
 5   children                        50000 non-null  object 
 6   meal                            50000 non-null  object 
 7   country                         49711 non-null  object 
 8   market_segment                  50000 non-null  object 
 9   distribution_channel            50000 non-null  object 
 10  is_repeated_guest               50000 non-null  int64  
 11  previous_cancellations          50000 non-null  int64  
 12  previous_bookings_not_canceled  

Contrairement au langage R qui permet via la commande glimpse de détecter automatiquement ce qui releève du string et du factor, en python nous travaillons avec object. 
dans une perspective d'utiliser ultérieuement des encoders, on reproduit l'approche tidymodels de R.


In [3]:
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype("category")
print(df.dtypes.head(10))

hotel                      category
lead_time                     int64
stays_in_weekend_nights       int64
stays_in_week_nights          int64
adults                        int64
children                   category
meal                       category
country                    category
market_segment             category
distribution_channel       category
dtype: object


On émule le glimpse de R

In [4]:
utils.glimpse(df)

Rows: 50000 | Columns: 23

hotel                     category   non-null: 50000  unique: 2      preview: ['City_Hotel', 'Resort_Hotel']
Categories (2, object): ['City_Hotel', 'Resort_Hotel']
lead_time                 int64      non-null: 50000  unique: 414    preview: [217   2  95 143 136]
stays_in_weekend_nights   int64      non-null: 50000  unique: 17     preview: [1 0 2 4 3]
stays_in_week_nights      int64      non-null: 50000  unique: 31     preview: [3 1 5 6 4]
adults                    int64      non-null: 50000  unique: 5      preview: [2 0 1 3 4]
children                  category   non-null: 50000  unique: 2      preview: ['none', 'children']
Categories (2, object): ['children', 'none']
meal                      category   non-null: 50000  unique: 5      preview: ['BB', 'HB', 'SC', 'Undefined', 'FB']
Categories (5, object): ['BB', 'FB', 'HB', 'SC', 'Undefined']
country                   category   non-null: 49711  unique: 154    preview: ['DEU', 'PRT', 'GBR', 'ROU', 'ESP']
Cat

In [5]:
# On sauvegarde dans le répertoire data/processed
# Sauvegarde du DataFrame en Excel
output_path = "../data/processed/hotels_clean.xlsx"

# index=False pour ne pas écrire l’index comme colonne supplémentaire
df.to_excel(output_path, index=False)

print(f"✅ Fichier enregistré dans {output_path}")


✅ Fichier enregistré dans ../data/processed/hotels_clean.xlsx


## Premère analyse exploratoire avec le jeu de données sauvegardé

In [6]:
# Import du dataset avec les facteurs

df_excel = pd.read_excel("../data/processed/hotels_clean.xlsx")

Matrice des enfants et proportion en python

In [8]:
# Comptage brut (volumes)
counts = df["children"].value_counts(dropna=False).sort_index()

# Proportions (fréquences relatives)
props = df["children"].value_counts(normalize=True, dropna=False).sort_index()

# Combiner en une matrice
children_matrix = pd.DataFrame({
    "count": counts,
    "proportion": props
})

children_matrix

,count,proportion
children,,
children,4038,0.08076
none,45962,0.91924


Première analyse :
Un chausse trape classique dans les jeux de données et la création de modèle, est la fameuse "class imbalance" ou déséquilibre de classes.
Dans ce cas, 92% des réservations ne contiennent pas d'enfants.

Ceci va forcément affecter les modèles de prédiction si ce n'est pas traiter. 

Plusieurs stratégies sont envisageables pour traiter les déséquilibres et nécessiteraient aussi de vérifier la distribution sur d'autres colonnes.
- Resampling : avec soit un oversampling de la minorite soit le contraire un undersampling. 
- Pondération de la perte de la classe minoritaire en régression logistique ou RandomForest
- Encoder aussi pour le traitement plus tard (0 ou 1) en feature engineering
- compléter si possible le jeu de données minoritaire, uniquement si c'est pertinent et réaliste


Avant de procéder à du resampling et du splitting, poursuivons notre exploration rapide du dataset.